In [ ]:
from model import FinancialLSTMModel
import pandas as pd

import os, random, numpy as np, tensorflow as tf

In [ ]:
CSV_PATH = './../data/GOOGL_1h.csv'
DATE_COL = 'Datetime'
TARGET = 'direction'
FEATURES = [
    [("Close", "minmax")],
    
    [("Close", "minmax"),
     ("rsi_14", "minmax"),
     ("rsi_28", "minmax"),
     ("rsi_7", "minmax"),],
    
    [("Close", "minmax"),
     ("macd", "standard"),
     ],
    
    [("Close", "minmax"),
     ("ema_20", "standard"),
     ("ema_50", "standard"),
     ],

    [("Close", "minmax"),
     ("stoch_k", "minmax"),
     ("stoch_d", "minmax"),
     ],

    [("Close", "minmax"),
     ("roc", "standard"),
     ],

    [("Close", "minmax"),
     ("adx", "minmax"),
     ("di_plus", "minmax"),
     ("di_minus", "minmax"),
     ],

    [("Close", "minmax"),
     ("atr", "standard"),
     ],

    [("Close", "minmax"),
     ("close_pos", "none"),
     ],


    [("Close", "minmax"),
     ("body_range_ratio", "none"),
     ],


    [("Close", "minmax"),   
     ("Volume", "minmax"),
     ],
]



SEQ_LENGTH = 60
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 100
TEST_RATIO = 0.1
VAL_SPLIT = 0.1


REPS = 10
OUTPUT_DIR='./../res/'

In [ ]:
# SEED = 42
# os.environ["PYTHONHASHSEED"]=str(SEED)
# os.environ["TF_DETERMINISTIC_OPS"]="1"
# os.environ["TF_CUDNN_DETERMINISTIC"]="1"
# random.seed(SEED)
# np.random.seed(SEED)
# tf.random.set_seed(SEED)

In [ ]:
res = []
for idx, feat in enumerate(FEATURES):
    for r in range(REPS):
        model = FinancialLSTMModel(
            csv_path=CSV_PATH,
            date_col=DATE_COL,
            features=feat,
            target="direction",
            seq_length=SEQ_LENGTH,
            batch_size=BATCH_SIZE,
            learning_rate=LEARNING_RATE,
            epochs=EPOCHS,
            test_ratio=TEST_RATIO,
            val_split=VAL_SPLIT,
            shuffle=False,
        )

        print(f" ---- Running with features: {feat}, repetition {r+1}/{REPS} ---- ")

        model.prepare_data()
        model.build_model()
        model.train()
        ev = model.evaluate()
        print(f" > Evaluation results: {ev}")
           
        res.append({
            "feature_set_index": idx,
            "features": [f[0] for f in feat],
            "repetition": r,
            **ev
        })
        
        df_res = pd.DataFrame(res)
        df_res.to_csv(os.path.join(OUTPUT_DIR, 'model_testing_feature_sets_results.csv'), index=False)
        
        